In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import random
random.seed(1228)
from sklearn.feature_extraction.text import *
from sklearn.metrics import *
from sklearn.pipeline import Pipeline
import re
from pymystem3 import Mystem
import numpy as np
import itertools
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from nltk.stem.snowball import RussianStemmer
import seaborn as sns



In [3]:
#1 analize

In [9]:
data = pd.read_csv("data_raw22.csv", sep=';')

In [10]:
data.head()

,id,rank,relevant,text
0,1,1,1,\tЯ очень расстроена! Пошли в межсерверную бит...
1,2,5,1,"\tУважаемые разработчики, благодарю за ваш отв..."
2,3,5,1,\tИграю с ноября. До сих пор не надоедает. Доб...
3,4,2,1,"\tУ меня Клара виснит,она бывает прозрачной ил..."
4,5,5,0,\tЛучшая игра!


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 4 columns):
id          214 non-null int64
rank        214 non-null int64
relevant    214 non-null int64
text        214 non-null object
dtypes: int64(3), object(1)
memory usage: 6.8+ KB


In [12]:
data['relevant'].value_counts()

1    119
0     95
Name: relevant, dtype: int64

In [15]:
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^А-Яа-я0-9(),!?@\'\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.lower()
    return df

data = standardize_text(data, "text")

data.to_csv("clean_data.csv")
data.head()

,id,rank,relevant,text
0,1,1,1,я очень расстроена! пошли в межсерверную битв...
1,2,5,1,"уважаемые разработчики, благодарю за ваш отве..."
2,3,5,1,играю с ноября до сих пор не надоедает доба...
3,4,2,1,"у меня клара виснит,она бывает прозрачной или..."
4,5,5,0,лучшая игра!


In [16]:
clean_data = pd.read_csv("clean_data.csv")
clean_data.tail()

,Unnamed: 0,id,rank,relevant,text
209,209,210,3,0,просят написать отзыв
210,210,211,5,1,"приятная игра иной раз сложная, но в меру д..."
211,211,212,5,1,интересная игра для людей всех возрастов спо...
212,212,213,5,0,игра очень нравится
213,213,214,1,1,"игра хорошая, но нервы нужны крепкие пока те..."


In [17]:
len_data = clean_data.text.apply(len)
len_data.describe()

count    214.000000
mean     147.551402
std      143.107710
min        6.000000
25%       31.250000
50%       97.500000
75%      222.500000
max      520.000000
Name: text, dtype: float64

In [89]:
from collections import Counter
from tqdm import tqdm_notebook as tqdm
cnt = Counter()
n_types = []
n_tokens = []
tokens = []
for index, row in tqdm(clean_data.iterrows(), total = len(clean_data)):
    tokens = row['text'].split()
    cnt.update(tokens)
    n_types.append(len(cnt))
    n_tokens.append(sum(cnt.values()))
for i in cnt.most_common(10):
    print(i)


('не', 179)
('и', 153)
('в', 103)
('игра', 87)
('я', 70)
('что', 65)
('а', 61)
('на', 61)
('за', 43)
('но', 43)


In [30]:
#2 preprocessing

In [221]:
#tokenization, lemmatisation, delete stop words

In [10]:
from nltk.tokenize import toktok
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer

import gensim
import string

/Applications/anaconda3/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [11]:
toktok = toktok.ToktokTokenizer()
print (toktok.tokenize(clean_data.text[3], return_str=True))

у меня клара виснит , она бывает прозрачной или вклеточку , а если на прогулке встретися то голова мужика влезает и так уже длится неделю , и у ирем бывает нет рук или головы , а бывает нет и головы и рук у не , да и сделайте чтобы покупки можно было совершать через оперетора 2 , надеюсь я буду услышана


In [12]:
print(word_tokenize(clean_data.text[3]))

['у', 'меня', 'клара', 'виснит', ',', 'она', 'бывает', 'прозрачной', 'или', 'вклеточку', ',', 'а', 'если', 'на', 'прогулке', 'встретися', 'то', 'голова', 'мужика', 'влезает', 'и', 'так', 'уже', 'длится', 'неделю', ',', 'и', 'у', 'ирем', 'бывает', 'нет', 'рук', 'или', 'головы', ',', 'а', 'бывает', 'нет', 'и', 'головы', 'и', 'рук', 'у', 'не', ',', 'да', 'и', 'сделайте', 'чтобы', 'покупки', 'можно', 'было', 'совершать', 'через', 'оперетора', '2', ',', 'надеюсь', 'я', 'буду', 'услышана']


In [13]:
def tokenize(text, tokenizer = toktok):
    try:
        return tokenizer.tokenize(text)
    except:
        return " "

In [233]:
#clean_data.text = clean_data.text.apply(tokenize)
#clean_data.head()

In [14]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

#print(tokenizer.tokenize(clean_data.text[3]))
#clean_data.text = clean_data["text"].apply(tokenizer.tokenize)
#clean_data.head()

In [57]:
from nltk.corpus import stopwords

In [90]:
mystopwords = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д', 
                                            'мой', 'очень', 'день', ',', 'просто', 'этот', 'вообще','что',
                                            'так', 'вот', 'быть', 'как', 'в', '—', '–', 'к', 'на', '...',]
def  remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return "ошибка"

In [91]:
clean_data.text = clean_data.text.apply(remove_stopwords) 
clean_data.head()

,Unnamed: 0,id,rank,relevant,text
0,0,1,1,1,"расстроена! пошли межсерверную битву, случился..."
1,1,2,5,1,"уважаемые разработчики, благодарю ваш ответ во..."
2,2,3,5,1,играю ноября сих пор надоедает добавляет интер...
3,3,4,2,1,"клара виснит,она бывает прозрачной вклеточку,а..."
4,4,5,5,0,лучшая игра!


In [ ]:
#tokinize + stowords

In [105]:
def tokenize_ru(file_text):
# firstly let's apply nltk tokenization
    tokens = word_tokenize(file_text)

# let's delete punctuation symbols
    tokens = [i for i in tokens if (i not in string.punctuation)]

# deleting stop_words
    stop_words = stopwords.words('russian')
    stop_words.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', '–', 'к', 'на', '...',
                       'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д', 'е',
                        'мой', 'очень', 'день', ',', 'просто', 'этот', 'вообще'])
    tokens = [i for i in tokens if (i not in stop_words)]

# cleaning words
    tokens = [i.replace("«", "").replace("»", "") for i in tokens]

    return tokens

In [107]:
clean_data['tokens'] = clean_data.text.apply(tokenize_ru)
clean_data.head()

,Unnamed: 0,id,rank,relevant,text,tokens
0,0,1,1,1,я очень расстроена! пошли в межсерверную битв...,"[расстроена, пошли, межсерверную, битву, случи..."
1,1,2,5,1,"уважаемые разработчики, благодарю за ваш отве...","[уважаемые, разработчики, благодарю, ваш, отве..."
2,2,3,5,1,играю с ноября до сих пор не надоедает доба...,"[играю, ноября, сих, пор, надоедает, добавляет..."
3,3,4,2,1,"у меня клара виснит,она бывает прозрачной или...","[клара, виснит, бывает, прозрачной, вклеточку,..."
4,4,5,5,0,лучшая игра!,"[лучшая, игра]"


In [94]:
from tqdm import tqdm_notebook as tqdm
cnt = Counter()
n_types = []
n_tokens = []
tokens = []
for index, row in tqdm(clean_data.iterrows(), total = len(clean_data)):
    tokens = row['text']
    cnt.update(tokens)
    n_types.append(len(cnt))
    n_tokens.append(sum(cnt.values()))
for i in cnt.most_common(10):
    print(i)


('игра', 103)
('игру', 29)
('играть', 17)
('время', 16)
('уровень', 16)
('уровни', 15)
('разработчики', 14)
('игре', 14)
('пройти', 14)
("''", 13)


In [ ]:
#tokinize + stowords

In [108]:
%%time 
from pymystem3 import Mystem

m = Mystem()
def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "

clean_data.text = clean_data.text.apply(lemmatize)

CPU times: user 26.2 ms, sys: 9.76 ms, total: 36 ms
Wall time: 1.42 s


In [109]:
text =  clean_data.text[3]
print(text)

у я клара виснять,она бывать прозрачный или вклеточка,а если на прогулка встретися то голова мужик влезать и так уже длиться неделя,и у ирема бывать нет рука или голова,а бывать нет и голова и рука у не ,да и сделать чтобы покупка можно быть совершать через оперетор     2, надеяться я быть услышать


In [68]:
print(str(clean_data.text[3]).strip('[]'))
print( ', '.join(map(str, clean_data.text[3])))

'клара', 'виснит', 'бывает', 'прозрачной', 'вклеточку', 'прогулке', 'встретися', 'голова', 'мужика', 'влезает', 'длится', 'неделю', 'ирем', 'бывает', 'рук', 'головы', 'бывает', 'головы', 'рук', 'сделайте', 'покупки', 'совершать', 'оперетора', '2', 'надеюсь', 'буду', 'услышана'
клара, виснит, бывает, прозрачной, вклеточку, прогулке, встретися, голова, мужика, влезает, длится, неделю, ирем, бывает, рук, головы, бывает, головы, рук, сделайте, покупки, совершать, оперетора, 2, надеюсь, буду, услышана


In [ ]:
#lemma true

In [24]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [95]:
def lemmas (text):
    tmp = []
    for w in text:
        word = morph.parse(w)[0].normal_form
        tmp.append(word)
    return tmp    

In [96]:
lemma = lemmas(clean_data.text[3])
print(lemma)

['клара', 'виснит', 'бывать', 'прозрачный', 'вклеточка', 'прогулка', 'встретися', 'голова', 'мужик', 'влезать', 'длиться', 'неделя', 'ирем', 'бывать', 'рука', 'голов', 'бывать', 'голов', 'рука', 'сделать', 'покупка', 'совершать', 'оперетор', '2', 'надеяться', 'быть', 'услышать']


In [110]:
clean_data.text = clean_data.tokens.apply(lemmas)

In [112]:
clean_data.head(15)

,Unnamed: 0,id,rank,relevant,text,tokens
0,0,1,1,1,"[расстроить, послать, межсерверный, битва, слу...","[расстроена, пошли, межсерверную, битву, случи..."
1,1,2,5,1,"[уважаемый, разработчик, благодарить, ваш, отв...","[уважаемые, разработчики, благодарю, ваш, отве..."
2,2,3,5,1,"[играть, ноябрь, сей, пора, надоедать, добавля...","[играю, ноября, сих, пор, надоедает, добавляет..."
3,3,4,2,1,"[клара, виснит, бывать, прозрачный, вклеточка,...","[клара, виснит, бывает, прозрачной, вклеточку,..."
4,4,5,5,0,"[хороший, игра]","[лучшая, игра]"
5,5,6,2,0,"[ужасный, игра, разработчик, смотреть, мнение,...","[ужасная, игра, разработчики, смотрят, мнения,..."
6,6,7,3,0,"[сказать, честно, игра, интернет, нравиться, м...","[скажу, честно, игра, интернете, нравится, мин..."
7,7,8,1,1,"[игра, напрямую, конвертировать, твой, время, ...","[игра, напрямую, конвертирующая, твое, время, ..."
8,8,9,3,0,"[игра, реклама, соответствовать]","[игра, рекламе, соответствует]"
9,9,10,5,1,"[игра, класный, играть, удовольствие, счёт, ал...","[игра, класная, играю, удовольствием, счет, ал..."


In [53]:
from tqdm import tqdm_notebook as tqdm
cnt = Counter()
n_types = []
n_tokens = []
tokens = []
for index, row in tqdm(clean_data.iterrows(), total = len(clean_data)):
    tokens = row['text']
    cnt.update(tokens)
    n_types.append(len(cnt))
    n_tokens.append(sum(cnt.values()))
for i in cnt.most_common(10):
    print(i)


('игра', 159)
('уровень', 52)
('играть', 36)
('время', 28)
('хотеть', 21)
('пройти', 20)
('разработчик', 18)
('сделать', 18)
('реклама', 18)
('который', 16)


In [ ]:
#lemma true

In [86]:
from collections import Counter

lemmata = []
for index, row in clean_data.iterrows():
    lemmata += row['text'].split()
cnt = Counter(lemmata)
for i in cnt.most_common(10):
    print(i)

('игра', 122)
('уровень', 36)
('играть', 27)
('время', 22)
('проходить', 20)
('хотеть', 19)
('игра,', 19)
('сделать', 17)
('который', 15)
('реклама', 13)


In [235]:
def stemming(text, stemmer = RussianStemmer()):
    try:
        return " ".join([stemmer.stem(w) for w in text.split()])
    except:
        return " "

In [236]:
#clean_data.text = clean_data.text.apply(stemming)

In [237]:
#clean_data.head()

In [ ]:
#vectorisation

In [ ]:
#bag of words

In [99]:
list_corpus = clean_data["text"].tolist()
list_labels = clean_data["relevant"].tolist()

In [114]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def cv(data):
    count_vectorizer = CountVectorizer(analyzer=callable, lowercase=False)
#analyzer='word', decode_error='replace', lowercase=False 
    emb = count_vectorizer.fit_transform(data)

    return emb, count_vectorizer

list_corpus = clean_data["text"].tolist()
list_labels = clean_data["relevant"].tolist()

X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, test_size=0.3, 
                                                                                random_state=40)

X_train_counts, count_vectorizer = cv(X_train)
X_test_counts = count_vectorizer.transform(X_test)

TypeError: 'bool' object is not iterable

In [77]:
text = clean_data.text[3]
lisst = []
lisst.append(text)
# create the transform
vectorizer = CountVectorizer()
# tokenize and build vocab
vectorizer.fit(lisst)
# summarize
print(vectorizer.vocabulary_)
# encode document
vector = vectorizer.transform(lisst)
# summarize encoded vector
print(vector.shape)
print(type(vector))
print(vector.toarray())

{'клара': 10, 'виснять': 2, 'она': 14, 'бывать': 0, 'прозрачный': 18, 'вклеточка': 3, 'прогулка': 17, 'встретися': 5, 'голова': 6, 'мужик': 11, 'влезать': 4, 'длиться': 8, 'неделя': 13, 'ирема': 9, 'рука': 19, 'да': 7, 'сделать': 20, 'покупка': 16, 'совершать': 21, 'оперетор': 15, 'надеяться': 12, 'быть': 1, 'услышать': 22}
(1, 23)
<class 'scipy.sparse.csr.csr_matrix'>
[[3 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1]]


In [ ]:
#tf idf

In [67]:
def tfidf(data):
    tfidf_vectorizer = TfidfVectorizer()

    emb = tfidf_vectorizer.fit_transform(data)

    return emb, tfidf_vectorizer

list_corpus2 = clean_data["text"].tolist()
list_labels2 = clean_data["relevant"].tolist()

X_train2, X_test2, y_train2, y_test2 = train_test_split(list_corpus2, list_labels2, test_size=0.2, 
                                                                                random_state=40)

X_train_counts2, count_vectorizer = tfidf(X_train)
X_test_counts2 = count_vectorizer.transform(X_test)

In [68]:
X_train_counts2

<149x1048 sparse matrix of type '<class 'numpy.float64'>'
	with 1992 stored elements in Compressed Sparse Row format>

In [ ]:
#3 training

In [81]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(C=30.0, class_weight='balanced', solver='newton-cg', 
                         multi_class='multinomial', n_jobs=-1, random_state=40)
clf.fit(X_train_counts, y_train)

y_predicted_counts = clf.predict(X_test_counts)

In [82]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

def get_metrics(y_test, y_predicted):  
    # true positives / (true positives+false positives)
    precision = precision_score(y_test, y_predicted, pos_label=None,
                                    average='weighted')             
    # true positives / (true positives + false negatives)
    recall = recall_score(y_test, y_predicted, pos_label=None,
                              average='weighted')
    
    # harmonic mean of precision and recall
    f1 = f1_score(y_test, y_predicted, pos_label=None, average='weighted')
    
    # true positives + true negatives/ total
    accuracy = accuracy_score(y_test, y_predicted)
    return accuracy, precision, recall, f1

accuracy, precision, recall, f1 = get_metrics(y_test, y_predicted_counts)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

accuracy = 0.800, precision = 0.809, recall = 0.800, f1 = 0.796


In [83]:
def get_most_important_features(vectorizer, model, n=5):
    index_to_word = {v:k for k,v in vectorizer.vocabulary_.items()}
    
    # loop for each class
    classes ={}
    for class_index in range(model.coef_.shape[0]):
        word_importances = [(el, index_to_word[i]) for i,el in enumerate(model.coef_[class_index])]
        sorted_coeff = sorted(word_importances, key = lambda x : x[0], reverse=True)
        tops = sorted(sorted_coeff[:n], key = lambda x : x[0])
        bottom = sorted_coeff[-n:]
        classes[class_index] = {
            'tops':tops,
            'bottom':bottom
        }
    return classes

importance = get_most_important_features(count_vectorizer, clf, 10)

In [85]:
importance

{0: {'tops': [(1.4071716211409298, 'наложница'),
   (1.4173555432273628, 'играть'),
   (1.4720274680678453, 'игра'),
   (1.5383315835289286, 'добавлять'),
   (1.642184398494618, 'мир'),
   (1.7925109341555532, 'па'),
   (1.8199381894462907, 'понравиться'),
   (1.830701498790468, 'причина'),
   (1.8523870574681092, 'приложение'),
   (2.114630011259471, 'скачать')],
  'bottom': [(-0.5273560343634993, 'смысл'),
   (-0.5587068417281483, 'крутой'),
   (-0.6193784035009469, 'взламывать'),
   (-0.6753714252759399, 'вообще'),
   (-0.6837148869861721, 'этот'),
   (-0.8101411085838699, 'создавать'),
   (-0.8244338019320747, 'соответствовать'),
   (-1.0715003433846142, 'класный'),
   (-1.077979311479725, 'любовь'),
   (-1.077979311479725, 'храм')]}}

In [88]:
clf2 = LogisticRegression(C=30.0, class_weight='balanced', solver='newton-cg', 
                         multi_class='multinomial', n_jobs=-1, random_state=40)
clf2.fit(X_train_counts2, y_train2)

y_predicted_counts2 = clf.predict(X_test_counts2)

In [89]:
accuracy2, precision2, recall2, f12 = get_metrics(y_test2, y_predicted_counts2)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy2, precision2, recall2, f12))

accuracy = 0.550, precision = 0.303, recall = 0.550, f1 = 0.390


In [91]:
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection, naive_bayes, svm
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#naive

In [118]:
clf_naive = naive_bayes.MultinomialNB()

In [119]:
param_dist_naive = {
             'alpha':[1, 0.1, 0.01, 0.001, 0.0001, 0.00001]
}

In [120]:
grid_naive = GridSearchCV(clf_naive, param_dist_naive, scoring='f1_weighted', cv=10, n_jobs=-1)
grid_naive.fit(X_train_counts, y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'alpha': [1, 0.1, 0.01, 0.001, 0.0001, 1e-05]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_weighted', verbose=0)

In [121]:
print(grid_naive.best_params_)
print(grid_naive.best_score_)
print(grid_naive.best_estimator_)

{'alpha': 1e-05}
0.8154680921671769
MultinomialNB(alpha=1e-05, class_prior=None, fit_prior=True)


In [122]:
y_pred_naive = grid_naive.predict(X_test_counts)

In [123]:
accuracy, precision, recall, f1 = get_metrics(y_test, y_pred_naive)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

accuracy = 0.625, precision = 0.736, recall = 0.625, f1 = 0.596


In [ ]:
#svc

In [112]:
clf_svc = svm.SVC()

In [113]:
param_dist_svc = {
             'C':[1,10,100,1000],
             'gamma':[1,0.1,0.001,0.0001], 
             'kernel':['linear','rbf','sigmoid']
}

In [114]:
grid_svc = GridSearchCV(clf_svc, param_dist_svc, scoring='f1_weighted', cv=10, n_jobs=-1)
grid_svc.fit(X_train_counts, y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'C': [1, 10, 100, 1000], 'gamma': [1, 0.1, 0.001, 0.0001], 'kernel': ['linear', 'rbf', 'sigmoid']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_weighted', verbose=0)

In [115]:
print(grid_svc.best_params_)
print(grid_svc.best_score_)
print(grid_svc.best_estimator_)

{'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
0.835221787436924
SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [116]:
y_pred_svc = grid_svc.predict(X_test_counts)

In [117]:
accuracy, precision, recall, f1 = get_metrics(y_test, y_pred_svc)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

accuracy = 0.850, precision = 0.850, recall = 0.850, f1 = 0.850


In [ ]:
#randomforest

In [92]:
clf_rf = RandomForestClassifier()

In [93]:
param_dist_rf = {
    'n_estimators': [5, 10, 20, 40],
    'criterion': ['gini', 'entropy'],
    'max_depth': [1, 3, 5, 7, 9, None],
    'min_samples_leaf': [1, 2, 4, 8, 16]
}

In [104]:
grid_forest = GridSearchCV(clf_rf, param_dist_rf, scoring='f1_weighted', cv=10, n_jobs=-1)
grid_forest.fit(X_train_counts, y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [5, 10, 20, 40], 'criterion': ['gini', 'entropy'], 'max_depth': [1, 3, 5, 7, 9, None], 'min_samples_leaf': [1, 2, 4, 8, 16]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_weighted', verbose=0)

In [105]:
print(grid_forest.best_params_)
print(grid_forest.best_score_)
print(grid_forest.best_estimator_)

{'criterion': 'entropy', 'max_depth': 7, 'min_samples_leaf': 2, 'n_estimators': 40}
0.8122284654415074
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=7, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [106]:
y_pred_rf = grid_forest.predict(X_test_counts)

In [107]:
accuracy, precision, recall, f1 = get_metrics(y_test, y_pred_rf)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

accuracy = 0.800, precision = 0.816, recall = 0.800, f1 = 0.800
